## Week 3 Part 3

Implement the non-local means algorithm. Try different window sizes. Add different levels of noise and see the influence of it in the need for larger or smaller neighborhoods. (Such block operations are easy when using Matlab, see for example the function at http://www.mathworks.com/help/images/ref/blockproc.html). Compare your results with those available in IPOL as demonstrated in the video lectures. http://www.ipol.im/pub/art/2011/bcm_nlm/
    

In [ ]:
from __future__ import division, print_function

import sys
import random
import time

import numpy as np
from matplotlib import pyplot as plt

import skimage
from skimage import img_as_float, img_as_ubyte
from skimage import io

%matplotlib inline

Some convenience functions.

In [ ]:
def add_noise(img, delta_max, delta_prob):
    """Adds uniform random noise to an image.
    """
    noise = np.random.randint(-delta_max,delta_max+1,size=img.shape)
    mask = np.random.rand(img.shape[0], img.shape[1])
    noise = noise * (mask < delta_prob) # keeps noise with prob < delta_prob
    im_noise = img + noise
    return np.clip(im_noise,0,255)

def add_salt_pepper(img, prob):
    """Adds salt and pepper noise to an image.
    """
    noisy_image = img.copy()
    noise = np.random.random(img.shape)
    noisy_image[noise > (1-prob/2)] = 255
    noisy_image[noise < prob/2] = 0
    return noisy_image

def nonlocal_pix_avg(img_blocks, block_blocks, rmse_threshold):
    pix_new = 0
    center_pix = int(img_blocks.shape[2]/2) + 1
    
    se = (img_blocks - block_blocks)**2
    sse = np.sum(np.sum(se, axis=2), axis=1)
    mse = sse/(se.shape[1]*se.shape[2])
    rmse = np.sqrt(mse)
    
    # print("rmse mean, sd, min, max: ", np.mean(rmse), np.std(rmse), np.min(rmse), np.max(rmse))

    mask = rmse < rmse_threshold
    
    pix_new = np.mean(img_blocks[mask, center_pix, center_pix]).astype(int)

    return pix_new

def img_as_block_array(img, win):
    """Rearranges an image into a 3D array of blocks.  
    Used for vectorized calculation of rmse."""
    rows = img.shape[0]
    cols = img.shape[1]
    img_blocks = np.zeros(((rows-win+1)*(cols-win+1),win,win))
    offset = np.floor(win/2).astype(int)
    
    idx = 0
    for r in range(offset, rows-offset):
        for c in range(offset, cols-offset):
            img_blocks[idx,:,:] = img[(r-offset):(r+offset+1),(c-offset):(c+offset+1)]
            idx=idx+1
            
    return img_blocks

def block_as_block_array(block, img_blocks):
    """Copies a given block along the first axis of a 3D array.
    Used for vectorized calculation of rmse"""
    block_blocks = np.zeros_like(img_blocks)
    rows_blocks = block_blocks.shape[1]
    cols_blocks = block_blocks.shape[2]
        
    for rb in range(rows_blocks):
        for cb in range(cols_blocks):
            block_blocks[:,rb,cb] = block[rb,cb]

    return block_blocks

def random_sample_img_blocks(img_blocks, n_blocks):
    """Random sample along the first axis of a 3D array"""
    n2 = img_blocks.shape[0]
    if n2 <= n_blocks:
        return img_blocks
    else:
        n = np.random.randint(0, n2, n_blocks) # sampling with replacement
        return img_blocks[n,:,:]

def nonlocal_avg(img, win, rmse_threshold, n_blocks):
    img_avg = np.zeros_like(img)
    rows = img_avg.shape[0]
    cols = img_avg.shape[1]
    offset = np.floor(win/2).astype(int)

    img_blocks = img_as_block_array(img, win)

    for r in range(offset, rows-offset):
        # sys.stdout.write(str(r)+", ") 

        for c in range(offset, cols-offset):
            block = img[(r-offset):(r+offset+1),(c-offset):(c+offset+1)]
        
            img_blocks_subset = random_sample_img_blocks(img_blocks, n_blocks)
            img_blocks_subset[0,:,:] = block # Make sure the target block is included.
        
            block_blocks = block_as_block_array(block, img_blocks_subset)

            img_avg[r,c] = nonlocal_pix_avg(img_blocks_subset, block_blocks, rmse_threshold)
    return img_avg

In [ ]:
def plot_1xc(imgs_list, titles_list, save_file=None):
    cols = len(imgs_list)
    i = 0
    
    fig, axes = plt.subplots(nrows=1, ncols=cols, figsize=(15,15))
    for c in range(cols):
        axes[c].imshow(imgs_list[i], cmap="gray")
        axes[c].set_title(titles_list[i], size=20)
        axes[c].set_xticks([])
        axes[c].set_yticks([])
        i = i + 1
    plt.tight_layout();
    
    if not (save_file == None):
        filename = save_file + time.strftime("%Y%m%d_%H%M") + ".png"
        fig.savefig(filename, bbox_inches='tight')

In [ ]:
sf = "None"
not (sf == None)

Main loop.

In [ ]:
img_original = io.imread("../lena512color.tiff")
img_original = img_original[:,:,1] # take the green channel as intensity
img_original = img_original[10:500, 10:500]

# img_noisy = add_salt_pepper(img_original, 0.01)
img_noisy = img_as_ubyte(skimage.util.random_noise(img_original))
plot_1xc([img_original, img_noisy], ["Original", "Gaussian Noise"])

In [ ]:
imgs_reconstructed = []
params = []
plot_it = True

# for win in [3, 5, 7, 9]: # Odd number only.
for win in [5]: # Odd number only.
    for rmse_threshold in range(20, 81, 20):
        # for n_blocks in [100, 400, 1600]:
        for n_blocks in [1600]:
            s = str(win) + "_" + str(rmse_threshold) + "_" + str(n_blocks) + "_" + time.strftime("%H:%M") + ", "
            sys.stdout.write(s) 
            params.append([win, rmse_threshold, n_blocks])
            imgs_reconstructed.append(nonlocal_avg(img_noisy, win, rmse_threshold, n_blocks))
            # imgs_reconstructed.append(img_original)

# Plotting
if plot_it:
    n_col=3
    n_row = np.ceil(len(params)/float(n_col)).astype(int)
    fig, axes = plt.subplots(nrows=n_row, ncols=n_col, figsize=(15,n_row*5))

    i = 0
    for r in range(n_row):
        for c in range(n_col):
            if i < len(params):
                axes[r,c].imshow(imgs_reconstructed[i], cmap="gray")
                axes[r,c].set_title("win, thresh, n:" + str(params[i]))
            i = i + 1
    plt.tight_layout()
    filename = "nonlocal_average_" + time.strftime("%Y%m%d_%H%M") + ".png"
    fig.savefig(filename, bbox_inches='tight')

Final output.

In [ ]:
img_original = io.imread("../lena512color.tiff")
img_original = img_original[:,:,1] # take the green channel as intensity
img_original = img_original[10:500, 10:500]

# img_noisy = add_salt_pepper(img_original, 0.01)
img_noisy = img_as_ubyte(skimage.util.random_noise(img_original))

img_reconstructed = nonlocal_avg(img_noisy, win=5, rmse_threshold=60, n_blocks=1600)

ims = [img_original, img_noisy, img_reconstructed]
titles = ["Original", "Gaussian Noise", "win=5, rmse=60, n=1600"]
plot_1xc(ims, titles, save_file="reconstructed")
